Kell egy mintakereső feltétel 8 MSPS-re

In [ ]:
# minta keresése
arg = list()
for i in range(len(iq_samples)-len(preamble_df)-1):
# az 1 esek nagyobbak mint a 0-k
    if (iq_samples[i] > iq_samples[i+4] and
        iq_samples[i+1] > iq_samples[i+5] and
        iq_samples[i+2] > iq_samples[i+6] and
        iq_samples[i+3] > iq_samples[i+7] and
        
        iq_samples[i+8] > iq_samples[i+12] and
        iq_samples[i+9] > iq_samples[i+13] and
        iq_samples[i+10] > iq_samples[i+14] and
        iq_samples[i+11] > iq_samples[i+15] and

        iq_samples[i+28] > iq_samples[i+27] and
        iq_samples[i+29] > iq_samples[i+26] and
        iq_samples[i+30] > iq_samples[i+25] and
        iq_samples[i+31] > iq_samples[i+24] and  

        iq_samples[i+36] > iq_samples[i+32] and
        iq_samples[i+37] > iq_samples[i+33] and
        iq_samples[i+38] > iq_samples[i+34] and
        iq_samples[i+39] > iq_samples[i+35] and

        iq_samples[i+64] > iq_samples[i+60] and
        iq_samples[i+65] > iq_samples[i+61] and
        iq_samples[i+66] > iq_samples[i+62] and
        iq_samples[i+67] > iq_samples[i+63] and

        iq_samples[i+99] > iq_samples[i+103] and
        iq_samples[i+98] > iq_samples[i+102] and
        iq_samples[i+97] > iq_samples[i+101] and
        iq_samples[i+96] > iq_samples[i+100]):
            arg.append(i)
len(arg)

Kell egy döntőfeltétel 8 MSPS-re

In [ ]:
# ki kellene listázni az összes 0 hibás üzenetet
# a minták az iq_samples[arg[i]:arg[i]+len(preamble_df)]
from crccheck.crc import Crc
nyers_rossz = []
uzenet = 0
data = 0
bin_data = 0
df_bits_string=ca_bits_string=icao_bits_string=me_bits_string=pi_bits_string = 0
messages = list()
raw_messages = list()
for i in range(len(arg)):
    if ((arg[i] + 960) > len(iq_samples)):
        continue
    start = arg[i]
    stop1 = start + math.floor(8*10**-6/Ts)
    PREAMBLE = iq_samples[start:stop1]
    #----------------DF mintái------------------#
    stop2 = stop1 + math.floor(5*10**-6/Ts)
    DF = iq_samples[stop1:stop2]
    #----------------CA mintái------------------#
    stop3 = stop2 + math.floor(3*10**-6/Ts)
    CA = iq_samples[stop2:stop3]
    #----------------ICAO mintái------------------#
    stop4 = stop3 + math.floor(24*10**-6/Ts)
    ICAO = iq_samples[stop3:stop4]
    #----------------ME mintái------------------#
    stop5 = stop4 + math.floor(56*10**-6/Ts)
    ME = iq_samples[stop4:stop5]
    #----------------PI mintái------------------#
    stop6 = stop5 + math.floor(24*10**-6/Ts)
    PI = iq_samples[stop5:stop6]
    full = np.concatenate([PREAMBLE,DF,CA,ICAO,ME,PI])
    bits = np.zeros(112,dtype = np.int64)
    full_bitek = full
    
    preamble_bits = full_bitek[:16*4]
    full_bitek = full_bitek[16*4:]
    nyers_rossz.append(full_bitek)
    raw_mess = list()
    #-------------------------8 MSPS-------------------------------#
    for n in range(112):
        if (sum(full_bitek[8*n:8*n+4]) < sum(full_bitek[8*n+4:8*n+8])):
            bits[n] = 0
            raw_mess.append(full_bitek[8*n:8*n+8])
        elif (sum(full_bitek[8*n:8*n+4]) > sum(full_bitek[8*n+4:8*n+8])):
            bits[n] = 1
        else:
            bits[n] = -1
    df_bits = bits[:5]
    #df_bits = ''.join(str(b) for b in df_bits.astype(int))
    ca_bits = bits[5:8]
    #ca_bits = ''.join(str(b) for b in ca_bits.astype(int))
    icao_bits = bits[8:32]
    #icao_bits = ''.join(str(b) for b in icao_bits.astype(int))
    me_bits = bits[32:88]
    pi_bits = bits[88:]
    type_code_bits = me_bits[:5]
    type_code_bits = ''.join(str(b) for b in type_code_bits.astype(int))
    #print(df_bits,ca_bits,icao_bits,me_bits,pi_bits,type_code_bits)
    ADS_B = Crc(width=24,poly=0x1FFF409,initvalue=0,xor_output=0,reflect_input=False,reflect_output=False)
    df_bits_string = ''.join(str(b) for b in df_bits)
    ca_bits_string = ''.join(str(b) for b in ca_bits)
    icao_bits_string = ''.join(str(b) for b in icao_bits)
    me_bits_string = ''.join(str(b) for b in me_bits)
    pi_bits_string = ''.join(str(b) for b in pi_bits)
    data = df_bits_string+ca_bits_string+icao_bits_string+me_bits_string+pi_bits_string
    bin_data = data
    data = hex(int(data, 2))[2:]
    if len(data) % 2 != 0:
        data = "0" + data
    
    data_ = bytes.fromhex(data)    
    err_mess = list()
    remainder = ADS_B.calc(data_)
    if (remainder == 0):
        messages.append(data)
        raw_messages.append(raw_mess)
        
    else:
        err_mess.append(data)
        raw_messages.append(raw_mess)
        
print(len(messages),len(err_mess))
print(err_mess[0])